In [2]:
import pandas as pd

In [4]:
sap_df = pd.read_csv('DB/exports/beli_reg_sap.csv')
sc_df = pd.read_csv('DB/exports/beli_reg_sc.csv')

In [5]:
sap_df

,profit_center,nama_outlet,posting_date,account_no,gl_acct_long_text,text,doc_id,doc_type,debit,kredit
0,BX08,KF.0225,1/11/2025,1107010301,Persediaan Barang Jadi,NaN,692000258,Z5,55000,55000
1,BX09,KF.0275,1/15/2025,1107010301,Persediaan Barang Jadi,NaN,692001123,Z5,414780,414780
2,BX50,KF Bhayangkara Kulon,1/12/2025,1107010301,Persediaan Barang Jadi,NaN,692000264,Z5,136000,136000
3,BX43,KF Sugiyopranoto Won,1/30/2025,1107010301,Persediaan Barang Jadi,NaN,692003012,Z5,111000,111000
4,BX03,KF.0070,1/21/2025,1107010301,Persediaan Barang Jadi,NaN,626008783,YQ,16132450,16132450
...,...,...,...,...,...,...,...,...,...,...
4015,BX52,KF Jamal 5.5,5/23/2025,1107010301,Persediaan Barang Jadi,NaN,692018443,Z5,266295,266295
4016,BX52,KF Jamal 5.5,5/30/2025,1107010301,Persediaan Barang Jadi,NaN,692019358,Z5,496124,496124
4017,BX52,KF Jamal 5.5,5/31/2025,1107010301,Persediaan Barang Jadi,NaN,692019618,Z5,232428,232428
4018,BX52,KF Jamal 5.5,5/5/2025,1107010301,Persediaan Barang Jadi,NaN,692015428,Z5,800990,800990


In [12]:
sap_df["posting_date"] = pd.to_datetime(sap_df["posting_date"])
sc_df["tgl_penerimaan"] = pd.to_datetime(sc_df["tgl_penerimaan"])

sap_df["outlet"] = sap_df["profit_center"]
sc_df["outlet"] = sc_df["kode_outlet"]

sap_df["tanggal"] = sap_df["posting_date"]
sc_df["tanggal"] = sc_df["tgl_penerimaan"]

sap_grouped = sap_df.groupby(["outlet", "tanggal"], as_index=False)["debit"].sum()
sc_grouped = sc_df.groupby(["outlet", "tanggal"], as_index=False)["jml_neto"].sum()

In [13]:
validasi = pd.merge(sap_grouped, sc_grouped, on=["outlet", "tanggal"], how="outer")
validasi["selisih"] = validasi["debit"] - validasi["jml_neto"]
validasi["status"] = validasi["selisih"].apply(lambda x: "VALID" if pd.isna(x) or abs(x) < 1 else "TIDAK VALID")

validasi.sort_values(by=["status", "outlet", "tanggal"], ascending=[False, True, True])


,outlet,tanggal,debit,jml_neto,selisih,status
0,BX01,2025-01-06,116000.0,116000.0,0.0,VALID
1,BX01,2025-01-08,1337666.0,1337666.0,0.0,VALID
5,BX01,2025-01-13,16383573.0,16383573.0,0.0,VALID
6,BX01,2025-01-14,340813.0,340813.0,0.0,VALID
9,BX01,2025-01-18,57989697.0,57989697.0,0.0,VALID
...,...,...,...,...,...,...
3001,BX51,2025-02-05,5212205.0,5212206.0,-1.0,TIDAK VALID
3023,BX51,2025-03-10,6050269.0,6050270.0,-1.0,TIDAK VALID
3051,BX51,2025-04-29,612158.0,612157.0,1.0,TIDAK VALID
3057,BX51,2025-05-08,34452564.0,34452563.0,1.0,TIDAK VALID
